### 1, 导入包及其他相关准备

In [1]:
import numpy as np
import os
import sys
from six.moves import cPickle as pickle
import tensorflow as tf
import time

In [2]:
f_path = './data/stock_group_1.pickle'
def load_data(f_path):
    data = None
    if os.path.exists(f_path):
        try:
            f = open(f_path, 'rb')
            data = pickle.load(f)
            f.close()
        except Exception as e:
            print('Unable to read data: ', e)
            raise
    else:
        print('Data file not exists.')
    return data

In [3]:
data = load_data(f_path)

In [5]:
data['train_dataset'].shape

(9442, 88)

In [10]:
data_train = data['train_dataset']
print(data_train.shape)
print(data_train.reshape(-1,88,1).shape)

(9442, 88)
(9442, 88, 1)


In [11]:
def get_batches(data, weights, labels, batch_size):
    data_len = len(data)
    for i in range(0, data_len, batch_size):
        end = i + batch_size
        if end > data_len:
            end = -1
        x = data[i: end].reshape(-1,data.shape[1],1)
        #print(x.shape)
        y = labels[i : end]
        w = weights[i: end]
        yield x, y, w

### 2, 构建模型

In [12]:
def build_inputs(num_features):
    '''
    构建输入
    '''
    inputs = tf.placeholder(tf.float32, [None, num_features, 1], name='inputs')
    targets = tf.placeholder(tf.float32, [None, 1], name='targets')
    weights = tf.placeholder(tf.float32, [None, 1], name='weights')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    return inputs, targets, weights, keep_prob

In [13]:
def resNet(inputs, num_features, layers, init_channel,filter_size, keep_prob, res_deep = 3, use_max_pool = False):
    '''
    残差网络
    '''
    layer_input = inputs
    layer_res = inputs
    channels = init_channel
    out_size = num_features
    
    padding = 'valid'
    if use_max_pool:
        padding = 'same'
    for i in range(1,layers + 1):
        C1 = tf.layers.conv1d(layer_input, 
                              channels,
                              filter_size,
                              padding=padding,
                              activation=tf.nn.elu,
                              kernel_initializer=tf.contrib.layers.xavier_initializer())
        layer_input = C1
        if not use_max_pool:
            out_size = out_size - filter_size + 1
        if i % res_deep == 0:
            channels = 2 * channels
            if not use_max_pool:
                C2 = tf.layers.conv1d(layer_res, 
                                      channels,
                                      (num_features - out_size) + 1,
                                      padding=padding,
                                      activation=tf.nn.elu,
                                      kernel_initializer=tf.contrib.layers.xavier_initializer())
                C3 = tf.layers.conv1d(layer_input, 
                                      channels,
                                      1,
                                      padding=padding,
                                      activation=tf.nn.elu,
                                      kernel_initializer=tf.contrib.layers.xavier_initializer())
                layer_input = C2 + C3
            else:
                layer_input = tf.layers.max_pooling1d((layer_res + C1), pool_size=4, strides=2, padding='same')
                layer_res = tf.layers.conv1d(layer_input, 
                                             channels,
                                             1,
                                             padding=padding,
                                             activation=tf.nn.elu,
                                             kernel_initializer=tf.contrib.layers.xavier_initializer())

    return layer_input, layer_input.get_shape().as_list()[1], layer_input.get_shape().as_list()[2]

In [14]:
def build_model(inputs, num_features, keep_prob, layers = 4, init_channel = 16, filter_size = 5):
    '''
    建立模型
    ''' 
    c1 = tf.layers.conv1d(inputs, init_channel, 1, padding='valid', 
                      activation=None, 
                      kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    layers_out, out_size, channels = resNet(c1, 
                                              num_features, 
                                              layers, 
                                              init_channel, 
                                              filter_size, 
                                              keep_prob,
                                              4,
                                              False)
    print('out size: ',out_size)
    print('channels: ',channels)
    layers_out_flat = tf.reshape(layers_out, (-1, out_size * channels))
    
    h = tf.layers.dense(layers_out_flat,256,activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    logits = tf.layers.dense(h, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer())
    
    return logits

In [15]:
#超参数
num_features = 88
batch_size = 1024
learning_rate = 0.0003
epochs = 10
kp = 0.75

###  3, 调用建模函数，建图

In [16]:
inputs, targets, weights, keep_prob = build_inputs(num_features)

logits = build_model(inputs, num_features, keep_prob)

out = tf.sigmoid(logits)

#回归weights
#targets_ = tf.sigmoid(weights)

entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=targets)
#entropy = tf.pow((logits - weights),2)

loss = tf.reduce_mean(entropy)

pre_out = tf.reduce_mean(tf.round(out))

correct_pred = tf.equal(tf.cast(tf.round(out), tf.int32), tf.cast(targets, tf.int32))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

out size:  72
channels:  32


In [17]:
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [19]:
save_every_n = 20000
saver = tf.train.Saver()
train_data = data['train_dataset']
train_w = data['train_w']
train_labels = data['train_labels']
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    counter = 0
    for e in range(epochs):
        for x, y, w in get_batches(train_data, train_w, train_labels, batch_size):
            counter += 1
            start = time.time()
            feed = {inputs: x,
                    targets: y,
                    keep_prob: kp,
                    weights: w}
            batch_loss, accuracy_, pre_out_, _ = sess.run([loss,accuracy,pre_out,opt], feed_dict=feed)
            valid_loss, pre_out_,valid_accuracy_= sess.run([loss,pre_out,accuracy], feed_dict={inputs:data['valid_dataset'].reshape(-1,num_features,1),
                                                    targets:data['valid_labels'],
                                                    keep_prob:1.0,
                                                    weights:data['valid_w']})
            end = time.time()
            print('Epoch: {}/{}. '.format(e+1, epochs),
                  'Train Step: {}. '.format(counter),
                  'pre_out: {:.4f}.'.format(pre_out_),
                  'Train acc: {:.4f}. '.format(accuracy_),
                  'Valid acc: {:.4f}. '.format(valid_accuracy_),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "./model/i{}.ckpt".format(counter))
    
    saver.save(sess, "./model/i{}.ckpt".format(counter))

Epoch: 1/10.  Train Step: 1.  pre_out: 0.5626. Train acc: 0.5146.  Valid acc: 0.4743.  0.3754 sec/batch
Epoch: 1/10.  Train Step: 2.  pre_out: 0.2925. Train acc: 0.5049.  Valid acc: 0.4545.  0.3495 sec/batch
Epoch: 1/10.  Train Step: 3.  pre_out: 0.2762. Train acc: 0.5283.  Valid acc: 0.5086.  0.3539 sec/batch
Epoch: 1/10.  Train Step: 4.  pre_out: 0.4074. Train acc: 0.5508.  Valid acc: 0.5283.  0.3501 sec/batch
Epoch: 1/10.  Train Step: 5.  pre_out: 0.4537. Train acc: 0.5557.  Valid acc: 0.5283.  0.3564 sec/batch
Epoch: 1/10.  Train Step: 6.  pre_out: 0.4708. Train acc: 0.5674.  Valid acc: 0.4940.  0.3561 sec/batch
Epoch: 1/10.  Train Step: 7.  pre_out: 0.5695. Train acc: 0.5518.  Valid acc: 0.4897.  0.3438 sec/batch
Epoch: 1/10.  Train Step: 8.  pre_out: 0.6304. Train acc: 0.5840.  Valid acc: 0.4889.  0.3508 sec/batch
Epoch: 1/10.  Train Step: 9.  pre_out: 0.6329. Train acc: 0.5801.  Valid acc: 0.4743.  0.3523 sec/batch
Epoch: 1/10.  Train Step: 10.  pre_out: 0.6055. Train acc: 0.560